In [1]:
import warnings
warnings.filterwarnings('ignore')

import sqlite3
conn = sqlite3.connect("ss.db")
cur  = conn.cursor()

import time
import os

import pandas as pd

qry = """SELECT bouncepoints_full.ID_Full, 
bouncepoints_full.Event, 
bouncepoints_full.Station, 
bouncepoints_full.Network, 
bouncepoints_full.Range, 
bouncepoints_full.MidLat, 
bouncepoints_full.MidLon, 
bouncepoints_full.EvtTime, 
bouncepoints_full.BAzimuth, 
downloads.Success
FROM bouncepoints_full
LEFT JOIN downloads 
ON bouncepoints_full.ID_Full=downloads.ID_Full
WHERE bouncepoints_full.MidLat > 0
and bouncepoints_full.MidLat < 75
and bouncepoints_full.MidLon < -60
and bouncepoints_full.MidLon > -135
and downloads.success == 'True' """

dfds = pd.read_sql_query(qry, conn, index_col=["ID_Full"])
dfds

,Event,Station,Network,Range,MidLat,MidLon,EvtTime,BAzimuth,Success
ID_Full,,,,,,,,,
C010490A.IU.HRV,C010490A,HRV,IU,108.909446,20.547609,-131.563087,1990-01-04T05:32:30.500000Z,266.227365,True
B010590D.II.ARU,B010590D,ARU,II,104.266518,69.370370,-88.045932,1990-01-05T18:27:00.200000Z,345.764029,True
B010590D.II.OBN,B010590D,OBN,II,100.280240,62.437544,-71.754853,1990-01-05T18:27:00.200000Z,325.031687,True
B010890B.II.NNA,B010890B,NNA,II,101.108863,27.430113,-109.450957,1990-01-08T19:44:38.300000Z,321.442603,True
B012090B.IU.HRV,B012090B,HRV,IU,109.175056,20.209490,-131.461033,1990-01-20T07:20:28.500000Z,265.830100,True
B021790A.G.SSB,B021790A,SSB,G,107.713395,32.345181,-64.631388,1990-02-17T01:22:16.500000Z,281.893241,True
B021790A.G.WUS,B021790A,WUS,G,143.150713,66.191930,-121.505477,1990-02-17T01:22:16.500000Z,8.639029,True
B021790A.II.ESK,B021790A,ESK,II,101.053694,35.761148,-73.437807,1990-02-17T01:22:16.500000Z,278.192464,True
B021790A.II.GAR,B021790A,GAR,II,145.486935,67.704887,-93.530982,1990-02-17T01:22:16.500000Z,353.675837,True


In [2]:
import obspy
from obspy.signal.filter import envelope
from scipy.signal import hilbert

from numpy import argmax, zeros

rbin = zeros(200)

snr ={}

for row in dfds.iterrows():
    bpindex = row[0]
    
    directory = 'data/mseed/%s/' % row[1].Event
    filename = directory + bpindex + '.mseed'
    st = obspy.read(filename).detrend().taper(0.05).filter('lowpass',freq=0.1,corners=4)
    
    tr = st[0]
    
    noise   = tr.copy().trim(tr.stats.starttime, tr.stats.starttime+100)
    signal  = tr.copy().trim(tr.stats.starttime+100, tr.stats.starttime+200)
    
    if len(noise.data) == 0 or len(signal.data) == 0:
        snr[bpindex] = -1.0
        continue
    
    snr[bpindex]    = max(abs(envelope(signal.data))) / max(abs(envelope(noise.data)))
    

In [3]:
snr.items()
dfout = pd.DataFrame(snr, index=["SNR"]).T
dfout

,SNR
B010102A.GE.DAG,0.728960
B010102A.GE.KBS,0.945076
B010102A.II.BORG,1.093976
B010102A.IU.KBS,0.952749
B010102A.IU.KEV,0.796581
B010102A.IU.KONO,1.314374
B010102A.XK.BADG,0.868132
B010102A.XK.BENH,0.880584
B010102A.XK.BOBR,1.183635
B010102A.XK.CASS,1.132945


In [4]:
#dfout.to_csv('snr.csv')
dfout.to_sql('snr',conn, if_exists="replace", index_label="ID_Full")

In [5]:
conn.close()